## Exercise: Grover's Algorithms
Brief description of Grover's algorithm (maybe on the slides?)

Package required:

In [ ]:
!pip install qibo
!pip install qibojit

Check the version

In [ ]:
import qibo
import qibojit
print(qibo.__version__)
print(qibojit.__version__)

Import qibo modules

In [ ]:
from qibo import models, gates, set_backend
set_backend("numpy")
from functions import *

### Exercise: Grover's algorithm using Qibo

We already saw in the slides that the Grover's algorithm is divided into 3 different parts
* Create superposition
* Create oracle
* Create diffusion operator

Assuming that we are looking to find the bistring 111...1 write the corresponding quantum circuit that implements
Grover's algorithm.

#### Step 1: creating the superposition
We need to write a circuit that will implement the superposition of all possible states.

In [ ]:
def create_superposition(nqubits):
    """Create circuit for superposition"""
    superposition = models.Circuit(nqubits+1)
    for i in range(nqubits):
        superposition.add(gates.H(i))
    superposition.add(gates.X(nqubits))
    superposition.add(gates.H(nqubits))

    return superposition
    


To check whether the circuit works as expected you can visualize the amplitudes using the function `plot_amplitude`.

In [ ]:
superposition = create_superposition(3)
plot_amplitudes(superposition())

### Coding the oracle
The oralcle is the operator that changes the sign of the amplitudes of the quantum states that encode solutions of the problem, in our case the state 11.....1


In [ ]:
def create_oracle(nqubits):
    """Oracle"""
    oracle = Circuit(nqubits+1)
    oracle.add(gates.X(nqubits).controlled_by(*range(nqubits)))
    return oracle

Lets check again using the `plot_amplitude` function

In [ ]:
amp = (create_superposition(3)+create_oracle(3))()
plot_amplitudes(amp)

### Coding the diffuser
To perform the diffusion operator we need to invert anything perpendicular to |s⟩. This can be done using a method similar to the Oracle.


In [ ]:
def create_diffuser(nqubits):
    diffuser = Circuit(nqubits+1)
    for i in range(nqubits):
        diffuser.add(gates.H(i))
    for i in range(nqubits):
        diffuser.add(gates.X(i))
    diffuser.add(gates.Z(0).controlled_by(*range(1,nqubits)))
    for i in range(nqubits):
        diffuser.add(gates.X(i))
    for i in range(nqubits):
        diffuser.add(gates.H(i))
    return diffuser

In [ ]:
amp = (create_superposition(3)+create_oracle(3)+create_diffuser(3))()
plot_amplitudes(amp)

In [ ]:
def grover_iterations(qubits, nsol=1):
    return int((np.pi/4)*np.sqrt((2**qubits)/nsol))

In [ ]:
def create_grover(nqubits, iterations):
    grover = models.Circuit(nqubits+1)
    superposition = create_superposition(nqubits)
    oracle = create_oracle(nqubits)
    diffuser = create_diffuser(nqubits)
    
    grover += superposition
    for _ in range(iterations):
        grover += oracle + diffuser
    grover.add([ gates.M(i) for i in range(nqubits)])
    return grover
        

In [ ]:
grover = create_grover(3, 1)

In [ ]:
plot_amplitudes(grover())

We can also have a look at the frequencies

In [ ]:
circuit = create_grover(5,3)

In [ ]:
result = circuit(nshots=1000)

In [ ]:
result.frequencies()

## Lets try to run the Grover's algorithm with an increasing number of qubits?

In [ ]:
# Try here

Lets run the following benchmark to understand why hardware acceleration is important.

In [ ]:
import time

def performance(backend, qubit_range, platform=None):
    if platform is None:
        set_backend(backend,)
    else:
        set_backend(backend, platform=platform)
    
    for i in qubit_range:
        iterations = grover_iterations(i,1)
    #     print(iterations)
        circuit = create_grover(i, iterations)
        print("Nqubits", i)
        start = time.time()
        result = circuit(nshots=1000)
        end = time.time()
        print(f"Time = {end-start}")
        print(result.frequencies())

In [ ]:
performance("numpy", range(4,20))

In [ ]:
performance("qibojit", range(4,20),"numba")

In [ ]:
performance("qibojit", range(4,20),"cupy")

In [ ]:
performance("qibojit", range(4,20),"cuquantum")